# RAG for Data Science interview preparation

In [1]:
!pip install sentence_transformer

ERROR: Could not find a version that satisfies the requirement sentence_transformer (from versions: none)
ERROR: No matching distribution found for sentence_transformer


In [512]:
from sentence_transformers import SentenceTransformer
import pandas as pd
from tqdm import auto

In [513]:
model = SentenceTransformer('all-mpnet-base-v2')

In [514]:
ds = pd.read_csv('data/db.csv')
ds

,section,question,answer
0,Classic_models,What are various ways to predict a binary resp...,"Things to look at: N, P, linearly seperable?, ..."
1,Classic_models,What methods for solving linear regression do ...,"To solve linear regression, you need to find t..."
2,Classic_models,What clustering algorithms do you know?,k-medoids: Takes the most central point instea...
3,Classic_models,How does DBScan work?,Two input parameters epsilon (neighborhood rad...
4,Classic_models,When would you choose K-means and when DBScan?,DBScan is more robust to noise.\r\nDBScan is b...
...,...,...,...
99,Statistics,Conjugate Prior,"According to Bayes’ theorem (6.23), the poster..."
100,Statistics,Are there any differences between the expected...,Expected value is used when we want to calcula...
101,Statistics,How do you identify if a coin is biased?,We collect data by flipping the coin 200 times...
102,Statistics,Exponential Family,An exponential family is a family of probabili...


In [515]:
ds['q+a'] = ds['question'] + ' ' + ds['answer']
ds

,section,question,answer,q+a
0,Classic_models,What are various ways to predict a binary resp...,"Things to look at: N, P, linearly seperable?, ...",What are various ways to predict a binary resp...
1,Classic_models,What methods for solving linear regression do ...,"To solve linear regression, you need to find t...",What methods for solving linear regression do ...
2,Classic_models,What clustering algorithms do you know?,k-medoids: Takes the most central point instea...,What clustering algorithms do you know? k-med...
3,Classic_models,How does DBScan work?,Two input parameters epsilon (neighborhood rad...,How does DBScan work? Two input parameters eps...
4,Classic_models,When would you choose K-means and when DBScan?,DBScan is more robust to noise.\r\nDBScan is b...,When would you choose K-means and when DBScan?...
...,...,...,...,...
99,Statistics,Conjugate Prior,"According to Bayes’ theorem (6.23), the poster...",Conjugate Prior According to Bayes’ theorem (6...
100,Statistics,Are there any differences between the expected...,Expected value is used when we want to calcula...,Are there any differences between the expected...
101,Statistics,How do you identify if a coin is biased?,We collect data by flipping the coin 200 times...,How do you identify if a coin is biased? We co...
102,Statistics,Exponential Family,An exponential family is a family of probabili...,Exponential Family An exponential family is a ...


In [516]:
ds = ds.reset_index().rename(columns={'index': 'id'})

In [517]:
stats = ds.to_dict('records')

Let's encode the field, containing question and answer, into vector, so we can use it for the search:

In [518]:
for record in auto.tqdm(stats):
    record['q+a_vector'] = model.encode(record["q+a"])

  0%|          | 0/104 [00:00<?, ?it/s]

In [519]:
import pickle

with open('data/ds.pkl', 'wb') as f:
    pickle.dump(stats, f)

In [536]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

es_client.info()

ObjectApiResponse({'name': 'd1afa5160e88', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'fkNVUkexQDeCEklNcE3vug', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [537]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "id": {"type": "keyword"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "answer": {"type": "text"},
            "q+a_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"},
        }
    }
}

In [542]:
index_name = "data_science_db"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'data_science_db'})

In [543]:
for doc in auto.tqdm(stats):
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

  0%|          | 0/104 [00:00<?, ?it/s]

In [545]:
es_client.indices.refresh(index=index_name)
es_client.cat.count(index=index_name, format="json")

ListApiResponse([{'epoch': '1723394861', 'timestamp': '16:47:41', 'count': '104'}])

In [546]:
search_term = "What is LDA?"
vector_search_term = model.encode(search_term)

In [547]:
query = {
    "field": "q+a_vector",
    "query_vector": vector_search_term,
    "k": 1,
    "num_candidates": 100, 
}

In [548]:
res = es_client.search(index=index_name, knn=query, source=["section", "question", "answer"])
res["hits"]["hits"]

[{'_index': 'data_science_db',
  '_id': 'D2JWQpEBffzaX7NOYPom',
  '_score': 0.79996276,
  '_ignored': ['q+a.keyword'],
  '_source': {'question': 'Linear Discriminant Analysis',
   'answer': 'Linear discriminant analysis is used as a tool for classification, dimension reduction, and data visualization. It has been around for quite some time now. Despite its simplicity, LDA often produces robust, decent, and interpretable classification results. When tackling real-world classification problems, LDA is often the benchmarking method before other more complicated and flexible ones are employed.\r\n',
   'section': 'Classic_models'}}]

In [549]:
API_TOKEN = '...'

In [131]:
!pip install -U --user "huggingface_hub[cli]"

In [132]:
!huggingface-cli login --token=$API_TOKEN

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\mary\.cache\huggingface\token
Login successful


In [133]:
import requests
headers = {"Authorization": f"Bearer {API_TOKEN}"}
API_URL = "https://api-inference.huggingface.co/models/mistralai/Mixtral-8x7B-Instruct-v0.1"

def query_mistral(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()
    
data = query_mistral({"inputs": """<s>[INST] I like drinking tea. [/INST] That's great to hear! Tea is a popular beverage...</s> [INST] What is the best way to brew tea? [/INST] """})

In [340]:
from huggingface_hub import InferenceClient

In [341]:
headers = {"Authorization": f"Bearer {API_TOKEN}"}

client = InferenceClient("mistralai/Mixtral-8x7B-Instruct-v0.1", headers=headers)

In [342]:
def llm(prompt, max_tokens=500):
    return client.text_generation(prompt=prompt, max_new_tokens=max_tokens)

In [1]:
# from transformers import AutoTokenizer, MixtralForCausalLM

In [3]:
# model = MixtralForCausalLM.from_pretrained("mistralai/Mixtral-8x7B-Instruct-v0.1")

In [91]:
# def llm(prompt, max_tokens=500):
#     model = MixtralForCausalLM.from_pretrained("mistralai/Mixtral-8x7B-Instruct-v0.1")
#     tokenizer = AutoTokenizer.from_pretrained("mistralai/Mixtral-8x7B-Instruct-v0.1")
#     inputs = tokenizer(prompt, return_tensors="pt")
#     # Generate
#     generate_ids = model.generate(inputs.input_ids, max_length=max_tokens)
#     return tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

Templates for prompts for our tasks:

In [343]:
templates = {
    'quiz': """[INST]
You're interviewing a user for a Data Scientist position. Ask a question using the QUESTION and the CONTEXT that you get. 
The answer to the question that you ask should be in the CONTEXT.
You can rephrase the QUESTION or ask something different 
Make it short, do not give tips, do notgive the answer and do not write notes 
Don't start with 'Based on the context' or 'Question', just shoot it right away

QUESTION: {question}
CONTEXT: 
{context}
[/INST]
""",
    'question': """[INST]
You're an assistant, helping to prepare for a Data Scientist interview. Answer the QUESTION based on the CONTEXT from the DS database. 
Use only the facts from the CONTEXT when answering the QUESTION. If you don't have an answer in CONTEXT, say it and stop. Don't write any additional information. 
Don't start with 'Based on'

QUESTION: {question}

CONTEXT: 
{context}
[/INST]
"""
}

In [344]:
import random

In [622]:
def get_body(mode, query=None):
    # answering the question mode
    if mode == 'question':
        body = {
            'knn': {
                "field": "q+a_vector",
                "k": 5,
                "num_candidates": 100, 
                "query_vector": query
                },
            '_source': ["section", "question", "answer"]
        }
    # asking user a question mode
    if mode == 'quiz':
        body = {
    'size': 1,
    '_source': ["section", "question", "answer", "id"],
    'query': 
    {
        "function_score": 
        {
            "random_score": 
                {
                    "field": 'id',
                    "seed": random.randint(0, 10000000)
                },
           },
        }
    }
    return body

In [617]:
def elastic_search(query, mode):
    vector_search_term = model.encode(query)
    body = get_body(mode, vector_search_term)
    response = es_client.search(index=index_name, body=body)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    return result_docs

In [618]:
def build_prompt(query, search_results, mode):
    prompt_template = templates[mode].strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['answer']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [614]:
def rag(query, mode='question'):
    search_results = elastic_search(query, mode)
    prompt = build_prompt(query, search_results, mode)
    answer = llm(prompt)
    return answer.strip()

Getting answers to our questions:

In [564]:
print(rag('Do we need a lot of data?'))

It depends on the situation. More data is not always better, especially if the data is biased or if the model has high bias. There is a trade-off between having more data and the additional storage, computational power, and memory it requires. It is also important to consider the quality of the data and the model being used. In the case of price prediction, data is not normally distributed and may contain skewed values or outliers, which would need to be pre-processed. Plotting the data before performing analysis can help identify these errors and outliers. Additionally, having too many features or dimensions can lead to overfitting and make it difficult to cluster observations.


In [619]:
print(rag('What is VIF?'))

A variance inflation factor (VIF) is a measure of the amount of multicollinearity in regression analysis. It estimates how much the variance of a regression coefficient is inflated due to multicollinearity. A VIF of three or below is not a cause for concern, and a VIF equal to one means variables are not correlated and multicollinearity does not exist in the regression model.


Quizzing the user:

In [620]:
print(rag('', 'quiz'))

How do AdaBoost and XGBoost differ in the way they update the ensemble of weak learners during the learning process?


In [621]:
print(rag('', 'quiz'))

When working with normalized predictor variables, if a weight is higher for one variable than another, can we conclude that the first variable is more significant in determining the output?
